# Data preparation

In [43]:
import os
import numpy as np
import pandas as pd

## Variables to dataframe
- df_{file}
- df_{prep_file}

In [44]:
challenge2025_dataset_path = "ETRI_lifelog_dataset/ch2025_data_items/"

file_names = [
    "mACStatus",
    "mScreenStatus",
    "mUsageStats",
    "mActivity",
    "mBle",
    "mWifi"
]

df_dict = {}
for name in file_names:
    file_path = os.path.join(challenge2025_dataset_path, f"ch2025_{name}.parquet")
    # 변수명 동적 생성
    globals()[f"df_{name}"] = pd.read_parquet(file_path)
    print(f"df_{name} loaded")

    

df_mACStatus loaded
df_mScreenStatus loaded
df_mUsageStats loaded
df_mActivity loaded
df_mBle loaded
df_mWifi loaded


In [45]:
df_mUsageStats.head()

,subject_id,timestamp,m_usage_stats
0,id01,2024-06-26 13:00:00,"[{'app_name': ' 캐시워크', 'total_time': 69}, {'ap..."
1,id01,2024-06-26 13:10:00,"[{'app_name': '통화', 'total_time': 26419}, {'ap..."
2,id01,2024-06-26 13:20:00,"[{'app_name': '메시지', 'total_time': 388651}, {'..."
3,id01,2024-06-26 13:30:00,"[{'app_name': '메시지', 'total_time': 211633}, {'..."
4,id01,2024-06-26 13:50:00,"[{'app_name': '카카오톡', 'total_time': 35446}, {'..."


## Train + Test

In [46]:
df_train = pd.read_csv('ETRI_lifelog_dataset/ch2025_metrics_train.csv')
df_test = pd.read_csv('ETRI_lifelog_dataset/ch2025_submission_sample.csv')

df_total = pd.concat([df_train, df_test], axis=0)

print("df_total.shape:", df_total.shape)
df_total.head()

df_total.shape: (700, 9)


,subject_id,sleep_date,lifelog_date,Q1,Q2,Q3,S1,S2,S3
0,id01,2024-06-27,2024-06-26,0,0,0,0,0,1
1,id01,2024-06-28,2024-06-27,0,0,0,0,1,1
2,id01,2024-06-29,2024-06-28,1,0,0,1,1,1
3,id01,2024-06-30,2024-06-29,1,0,1,2,0,0
4,id01,2024-07-01,2024-06-30,0,1,1,1,1,1


## df_merge

In [47]:
import pandas as pd

# lifelog_date가 string이면 datetime으로 변환
df_total['lifelog_date'] = pd.to_datetime(df_total['lifelog_date'])

result_list = []

for sid, group in df_total.groupby('subject_id'):
    # 각 id별 min/max lifelog_date
    min_date = group['lifelog_date'].min()
    max_date = group['lifelog_date'].max()
    
    # 10분 단위로 timestamp 생성
    timestamps = pd.date_range(start=min_date, end=max_date + pd.Timedelta(days=1) - pd.Timedelta(minutes=10), freq='10min')
    # 위 코드에서 max_date + 1일 - 10분 까지 하는 이유는 23:50:00 포함시키기 위해서 (끝나는 날짜 23:50까지 포함)

    # DataFrame 생성
    df_id = pd.DataFrame({
        'subject_id': sid,
        'timestamp': timestamps
    })
    result_list.append(df_id)

# 모든 id에 대해 concat
df_merge = pd.concat(result_list, ignore_index=True)

print(df_merge)
print(df_merge.shape)

       subject_id           timestamp
0            id01 2024-06-26 00:00:00
1            id01 2024-06-26 00:10:00
2            id01 2024-06-26 00:20:00
3            id01 2024-06-26 00:30:00
4            id01 2024-06-26 00:40:00
...           ...                 ...
122827       id10 2024-09-26 23:10:00
122828       id10 2024-09-26 23:20:00
122829       id10 2024-09-26 23:30:00
122830       id10 2024-09-26 23:40:00
122831       id10 2024-09-26 23:50:00

[122832 rows x 2 columns]
(122832, 2)


### df_prep_mUsageStats

In [48]:
df_mUsageStats['m_usage_stats'][1]

array([{'app_name': '통화', 'total_time': 26419},
       {'app_name': '토스', 'total_time': 119896},
       {'app_name': '전화', 'total_time': 59284},
       {'app_name': '카카오톡', 'total_time': 6744},
       {'app_name': 'NAVER', 'total_time': 67042},
       {'app_name': '\xa0✝️성경일독Q', 'total_time': 1504},
       {'app_name': 'One UI 홈', 'total_time': 209417}], dtype=object)

In [49]:
# 'subject_id', 'timestamp' 열만 추출
df_prep_mUsageStats = df_mUsageStats[['subject_id', 'timestamp']].copy()

# 각 행에 대해 total_time의 합을 구하는 함수
def sum_usage_time(usage_stats):
    #usage_stats: list of dict
    sum_time = 0
    for usage in usage_stats:
        # 'usage_time'이 없으면 0으로 처리
        sum_time += usage.get('total_time', 0)
    return sum_time

df_prep_mUsageStats['m_usage_time'] = df_mUsageStats['m_usage_stats'].apply(sum_usage_time)

# 결과 출력 (앞 10개)
print(df_prep_mUsageStats.head(3))


  subject_id           timestamp  m_usage_time
0       id01 2024-06-26 13:00:00          7955
1       id01 2024-06-26 13:10:00        490306
2       id01 2024-06-26 13:20:00        599985


### df_prep_mActivity

In [50]:
import pandas as pd

# 1. 'subject_id', 'timestamp'만 추출
df_prep_mActivity = df_mActivity[['subject_id', 'timestamp']].copy()

# 2. met_activity 열 선언
df_prep_mActivity['met_activity'] = 0.0

# 3. 매핑 함수와 시간대별 가중치 함수 정의
activity_to_met = {0: 1.3, 1: 7.2, 2: 2.3, 3: 1.1, 4: 1.0, 5: 1.3, 7: 3.4, 8: 8.0}

def get_time_weight(ts):
    hour = ts.hour
    if 0 <= hour < 8:
        return 0.3
    elif 8 <= hour < 18:
        return 0.7
    else:
        return 1.0

# timestamp를 datetime으로 변환
df_prep_mActivity['timestamp'] = pd.to_datetime(df_prep_mActivity['timestamp'])

# 4. 계산하여 met_activity 값 입력
def calc_weighted_met(row):
    met = activity_to_met.get(df_mActivity.loc[row.name, 'm_activity'], 1.0)  # 매핑 없으면 1.0
    weight = get_time_weight(row['timestamp'])
    return met * weight

df_prep_mActivity['met_activity'] = df_prep_mActivity.apply(calc_weighted_met, axis=1)

# 결과 확인 (head 10)
print(df_prep_mActivity.tail(10))


       subject_id           timestamp  met_activity
961052       id10 2024-09-26 23:50:00           1.1
961053       id10 2024-09-26 23:51:00           1.1
961054       id10 2024-09-26 23:52:00           1.1
961055       id10 2024-09-26 23:53:00           1.1
961056       id10 2024-09-26 23:54:00           1.1
961057       id10 2024-09-26 23:55:00           1.1
961058       id10 2024-09-26 23:56:00           1.1
961059       id10 2024-09-26 23:57:00           1.1
961060       id10 2024-09-26 23:58:00           1.1
961061       id10 2024-09-26 23:59:00           1.1


### df_prep_mBle

In [51]:
df_mBle['m_ble'][2]

array([{'address': '04:F5:AE:39:95:E0', 'device_class': '0', 'rssi': -44},
       {'address': '0D:BE:52:E6:13:00', 'device_class': '0', 'rssi': -80},
       {'address': '0F:13:09:75:36:FE', 'device_class': '0', 'rssi': -76},
       {'address': '15:61:31:49:2F:F5', 'device_class': '0', 'rssi': -90},
       {'address': '2B:70:D0:E0:3C:84', 'device_class': '0', 'rssi': -83},
       {'address': '2F:EF:C3:70:A0:97', 'device_class': '0', 'rssi': -41},
       {'address': '30:EF:FE:9E:E4:AD', 'device_class': '0', 'rssi': -70},
       {'address': '38:54:47:EA:74:E1', 'device_class': '0', 'rssi': -61},
       {'address': '38:C8:8D:5C:AD:83', 'device_class': '0', 'rssi': -71},
       {'address': '40:BC:AF:DD:04:C5', 'device_class': '0', 'rssi': -92},
       {'address': '45:33:4C:24:C4:C9', 'device_class': '0', 'rssi': -83},
       {'address': '47:F1:F3:8D:95:20', 'device_class': '0', 'rssi': -88},
       {'address': '54:15:89:95:27:44', 'device_class': '1064', 'rssi': -75},
       {'address': '54

In [52]:
# 'subject_id', 'timestamp' 열만 추출
df_prep_mBle = df_mBle[['subject_id', 'timestamp']].copy()

# 각 행에 대해 total_time의 합을 구하는 함수
def sum_ble_rssi(ble_stats):
    #ble_stats: list of dict
    sum_rssi = 0
    for ble in ble_stats:
        # 'ble'이 없으면 0으로 처리
        sum_rssi += np.exp(ble.get('rssi', 0) / 10)
    return sum_rssi

df_prep_mBle['m_wtb_rssi'] = df_mBle['m_ble'].apply(sum_ble_rssi)

# 결과 출력 (앞 10개)
print(df_prep_mBle.head(3))
print(df_prep_mBle.shape)


  subject_id           timestamp  m_wtb_rssi
0       id01 2024-06-26 12:13:00    0.102155
1       id01 2024-06-26 12:23:00    0.098621
2       id01 2024-06-26 12:33:00    0.037712
(21830, 3)


### df_prep_mWifi

In [53]:

# 'subject_id', 'timestamp' 열만 추출
df_prep_mWifi = df_mWifi[['subject_id', 'timestamp']].copy()

# 각 행에 대해 total_time의 합을 구하는 함수
def sum_wifi_rssi(wifi_stats):
    # wifi_stats: list of dict
    sum_rssi = 0
    for wifi in wifi_stats:
        # 'rssi'가 없으면 0으로 처리
        sum_rssi += np.exp(wifi.get('rssi', 0) / 10)
    return sum_rssi

df_prep_mWifi['m_wtw_rssi'] = df_mWifi['m_wifi'].apply(sum_wifi_rssi)

# 결과 출력 (앞 10개)
print(df_prep_mWifi.head(3))
print(df_prep_mWifi.shape)


  subject_id           timestamp  m_wtw_rssi
0       id01 2024-06-26 12:03:00    0.202476
1       id01 2024-06-26 12:13:00    0.091135
2       id01 2024-06-26 12:23:00    0.063361
(76336, 3)


In [54]:
# 1. df_merge에 m_usage_time 열 추가 (일단 0으로 초기화)
df_merge['m_usage_time'] = 0.0

# 2. df_prep_mUsageStats의 timestamp를 datetime으로 변환
df_prep_mUsageStats['timestamp'] = pd.to_datetime(df_prep_mUsageStats['timestamp'])

# 3. subject_id별로 병합 및 분할
for sid, sub_df in df_prep_mUsageStats.groupby('subject_id'):
    # 해당 subject_id의 10분 단위 timestamp만 필터
    merge_idx = df_merge['subject_id'] == sid
    sub_merge = df_merge.loc[merge_idx].copy()
    
    # mUsageStats가 있는 시간 리스트(보통 30분 단위 등)
    time_list = sub_df['timestamp'].tolist()
    
    for i, t in enumerate(time_list):
        # 시작, 끝 시간 구간
        start_time = t
        if i < len(time_list) - 1:
            end_time = time_list[i+1]
        else:
            # 마지막 구간은 다음 30분(혹은 원하는 범위)까지
            end_time = t + pd.Timedelta(minutes=30)  
        
        # 구간 내 10분 단위 timestamp 모두 구하기
        time_range = pd.date_range(start=start_time, end=end_time - pd.Timedelta(minutes=10), freq='10min')
        n = len(time_range)
        # m_usage_time을 10분 단위로 분할
        per_time = sub_df.iloc[i]['m_usage_time'] / n if n > 0 else 0
        
        # df_merge에서 해당 subject, timestamp 매칭하여 값 입력
        cond = (df_merge['subject_id'] == sid) & (df_merge['timestamp'].isin(time_range))
        df_merge.loc[cond, 'm_usage_time'] = per_time

# 4. 결과 확인
print(df_merge.head(20))


   subject_id           timestamp  m_usage_time
0        id01 2024-06-26 00:00:00           0.0
1        id01 2024-06-26 00:10:00           0.0
2        id01 2024-06-26 00:20:00           0.0
3        id01 2024-06-26 00:30:00           0.0
4        id01 2024-06-26 00:40:00           0.0
5        id01 2024-06-26 00:50:00           0.0
6        id01 2024-06-26 01:00:00           0.0
7        id01 2024-06-26 01:10:00           0.0
8        id01 2024-06-26 01:20:00           0.0
9        id01 2024-06-26 01:30:00           0.0
10       id01 2024-06-26 01:40:00           0.0
11       id01 2024-06-26 01:50:00           0.0
12       id01 2024-06-26 02:00:00           0.0
13       id01 2024-06-26 02:10:00           0.0
14       id01 2024-06-26 02:20:00           0.0
15       id01 2024-06-26 02:30:00           0.0
16       id01 2024-06-26 02:40:00           0.0
17       id01 2024-06-26 02:50:00           0.0
18       id01 2024-06-26 03:00:00           0.0
19       id01 2024-06-26 03:10:00       

In [55]:
# 1. df_prep_mActivity의 timestamp를 datetime으로 변환
df_prep_mActivity['timestamp'] = pd.to_datetime(df_prep_mActivity['timestamp'])
df_prep_mActivity = df_prep_mActivity.sort_values(['subject_id', 'timestamp'])

result_list = []

for sid, group in df_prep_mActivity.groupby('subject_id'):
    group = group.reset_index(drop=True)
    # 1분 단위 타임라인 생성 (최소~최대)
    min_time = group['timestamp'].min()
    max_time = group['timestamp'].max()
    timeline = pd.DataFrame({'timestamp': pd.date_range(start=min_time, end=max_time, freq='1min')})
    timeline['subject_id'] = sid
    timeline['met_activity'] = 0.0

    # 각 구간별로 분배
    for i in range(len(group)):
        curr_time = group.loc[i, 'timestamp']
        curr_met = group.loc[i, 'met_activity']

        # 현재 값은 그대로 할당
        timeline.loc[timeline['timestamp'] == curr_time, 'met_activity'] += curr_met

        # 마지막 값이 아니면, 다음 값까지 분할
        if i < len(group) - 1:
            next_time = group.loc[i+1, 'timestamp']
            minutes = int((next_time - curr_time).total_seconds() // 60)
            if minutes > 0:
                value_per_minute = group.loc[i+1, 'met_activity'] / minutes
                mask = (timeline['timestamp'] > curr_time) & (timeline['timestamp'] <= next_time)
                timeline.loc[mask, 'met_activity'] += value_per_minute

    result_list.append(timeline)

# 모든 subject의 1분 단위 met_activity 시계열 데이터프레임
df_mActivity_min = pd.concat(result_list, ignore_index=True)

# 10분 단위로 합산해서 df_merge에 합치기
# df_merge는 subject_id, timestamp(10분 단위)로 구성되어 있다고 가정
def sum_met(sub_df, start, end):
    mask = (sub_df['timestamp'] >= start) & (sub_df['timestamp'] < end)
    return sub_df.loc[mask, 'met_activity'].sum()

# 결과 저장할 열 추가
df_merge['met_activity'] = 0.0

for sid in df_merge['subject_id'].unique():
    min_df = df_mActivity_min[df_mActivity_min['subject_id'] == sid]
    for idx, row in df_merge[df_merge['subject_id'] == sid].iterrows():
        start = row['timestamp']
        end = start + pd.Timedelta(minutes=10)
        total_met = sum_met(min_df, start, end)
        df_merge.at[idx, 'met_activity'] = total_met

# 결과 확인
print(df_merge.head(20))


   subject_id           timestamp  m_usage_time  met_activity
0        id01 2024-06-26 00:00:00           0.0           0.0
1        id01 2024-06-26 00:10:00           0.0           0.0
2        id01 2024-06-26 00:20:00           0.0           0.0
3        id01 2024-06-26 00:30:00           0.0           0.0
4        id01 2024-06-26 00:40:00           0.0           0.0
5        id01 2024-06-26 00:50:00           0.0           0.0
6        id01 2024-06-26 01:00:00           0.0           0.0
7        id01 2024-06-26 01:10:00           0.0           0.0
8        id01 2024-06-26 01:20:00           0.0           0.0
9        id01 2024-06-26 01:30:00           0.0           0.0
10       id01 2024-06-26 01:40:00           0.0           0.0
11       id01 2024-06-26 01:50:00           0.0           0.0
12       id01 2024-06-26 02:00:00           0.0           0.0
13       id01 2024-06-26 02:10:00           0.0           0.0
14       id01 2024-06-26 02:20:00           0.0           0.0
15      

In [64]:
df_merge.head()

,subject_id,timestamp,m_usage_time,met_activity,m_wtb_rssi,m_wtw_rssi
0,id01,2024-06-26 00:00:00,0.0,0.0,NaN,NaN
1,id01,2024-06-26 00:10:00,0.0,0.0,NaN,NaN
2,id01,2024-06-26 00:20:00,0.0,0.0,NaN,NaN
3,id01,2024-06-26 00:30:00,0.0,0.0,NaN,NaN
4,id01,2024-06-26 00:40:00,0.0,0.0,NaN,NaN
